**MEMBROS DO GRUPO**

Gabriel Meirelles Carvalho Orlando 790728

Giullio Emmanuel 790965

Jorge Luiz Medeiros Pires 790942

Lucas Martins de Almeida 758983

Pedro Gonçalves dos Santos 794042

# Setup


Para rodar os códigos e verificar a eficácia dos algoritmos, é necessário baixar os arquivos de input direto do GitHub

In [24]:
!wget /content/ https://raw.githubusercontent.com/PedroGSantos/Median-of-a-Set/main/inputs/input.in

/content/: Scheme missing.
--2024-01-28 15:34:23--  https://raw.githubusercontent.com/PedroGSantos/Median-of-a-Set/main/inputs/input.in
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17 [text/plain]
Saving to: ‘input.in’

input.in            100%[===================>]      17  --.-KB/s    in 0s      

2024-01-28 15:34:23 (1.34 MB/s) - ‘input.in’ saved [17/17]

FINISHED --2024-01-28 15:34:23--
Total wall clock time: 0.06s
Downloaded: 1 files, 17 in 0s (1.34 MB/s)


# Enunciado


**1 Median of a set**

The median is described as the number separating the higher half of a sample, a population, or a probability distribution, from the lower half. The median of a finite list of numbers can be found by arranging all the observations from lowest to highest value and picking the middle one. If there is an even number of observations, the median is the mean of the two middle values.

For example:

*   1,2,3,4,5 : The median is 3.

*   1,5,9,2,8,4: The median is 4.5.


Compute the median of a given set of values.

# Ambiente de teste utilizado

Para a realização dos nosso experimentos, utilizamos um servidor externo, já que o Google Collab apresenta diversas instabilidade e possui apenas uma GPU o que limita um pouco os nossos testes. Dessa maneira utilizamos um servidor, que para funcionar utiliza um sistema de docker e  está na UFSCAR, sendo gerenciado pelo Professor Hermes, e tem as seguintes configurações:

💡 RAM: 128 GB

💡 Processor: 2 Intel(R) Xeon(R) Silver 4208 CPU

💡 GPU: 3 NVIDIA GeForce RTX 2080 Super

💡 Ubuntu 22.04

💡 CUDA 12.0

💡 Docker Version: 20.10.21

# Código base sequencial


O código abaixo consiste em uma implementação oficial da Maratona de Programação Paralela para o cálculo da mediana. Ele não utiliza ordenação para descobrir a mediana, mas sim uma contagem de frequência dos valores através dos índices de outro array. Através dessa contagem, conseguimos passar por esse array auxiliar até chegar no elemento central, fornecendo o valor central, caso a quantidade de números seja impar, ou somando com o próximo elemento e dividindo por 2, caso a quantidade de números seja par.

In [1]:
%%writefile seq_median_oficial.cpp

#include <iostream>
#include <fstream>

typedef uint uint;
const uint MAX_VALUE = 1<<20;

int main(int argc, char* argv[]) {
     uint n_seen = 0;
     uint value;
     uint count[MAX_VALUE];

     // Initializes the array
     for(uint i = 0; i < MAX_VALUE; ++i) {
          count[i] = 0;
     }

     // Reads the input file
     std::ifstream input(argv[1]);
     input >> value;
     while (!input.eof()) {
          ++n_seen;
          ++count[value];  // Counts the ocurrence of a number
          input >> value;
     }
     input.close();

     // Open the output file
     std::ofstream output("problem_output");

     // Searching for the median
     uint half_seen = n_seen/2;
     // This algorithm assumes a odd numbered set. If it is even, then we
     // can "remove" the last element, making the set odd without altering
     // the results. This will be done subtracting 1 from the hals_seen.
     // Eventually we will restore the value removed and fix our median.
     if (n_seen % 2 == 0) --half_seen;
     uint median = 0;
     uint seen_until_median = count[0];
     while (seen_until_median <= half_seen && median < MAX_VALUE) {
          ++median;
          seen_until_median = seen_until_median + count[median];
     };

     if (n_seen % 2) {
          output << median << std::endl;
     } else {
          double mean_median = median;
          ++median;
          while (count[median] == 0) ++median;
          mean_median = (mean_median+median)/2.0;
          output << mean_median << std::endl;
     }
     output.close();

}

Writing seq_median_oficial.cpp


Utilizando um arquivo de input e o utilitário ***time***, podemos observar o tempo de execução desse algoritmo de contagem. No teste abaixo, o algoritmo é executado 3 vezes, visto que, em programação paralela, é essencial pegarmos uma média do tempo em várias execuções diferentes.

In [13]:
%%writefile run_seq_oficial.sh

#!/bin/bash

# Compilando
! g++ seq_median_oficial.cpp -o seq_median_oficial

for j in {1..3}; do
  echo
  echo "run $j"
  time ./seq_median_oficial input.in > /dev/null
done


Writing run_seq_oficial.sh


In [14]:
! chmod +x run_seq_oficial.sh;
! ./run_seq_oficial.sh


run 1

real	0m0.011s
user	0m0.005s
sys	0m0.006s

run 2

real	0m0.012s
user	0m0.005s
sys	0m0.007s

run 3

real	0m0.011s
user	0m0.005s
sys	0m0.006s


Partindo para uma primeira análise, podemos observar que existem soluções que utilizam ordenação para descobrir a mediana de um conjunto de dados, logo, seria uma possível implementação com melhor desempenho sequencial. O código abaixo consiste em uma implementação usando a primitiva sort do cpp, que é muito otimizada e consegue lidar com grandes conjuntos de dados, dessa maneira a mediana esperada é realmente 5.5


In [16]:
%%writefile seq_median_sort.cpp

#include <algorithm>
#include <cstdlib>
#include <iostream>
#include <stdlib.h>
#include <fstream>

int main( int argc, char *argv[]) {
  uint value;

  std::vector<uint> vec;


  // Reads the input file
  std::ifstream input(argv[1]);
  input >> value;
  while (!input.eof()) {
      vec.push_back(value);
      input >> value;
  }
  input.close();

  // sort data on the device (846M keys per second on GeForce GTX 480)
  std::sort(vec.begin(), vec.end());

  double median = 0;

  if (vec.size() % 2 == 1){
   median = vec[(vec.size()-1)/2] ;
  }

  else
    median = (vec[(vec.size()/2)-1] + vec[vec.size()/2]) / 2.0;



    printf("The median is %.3lf\n",median);

  return 0;
}


Writing seq_median_sort.cpp


Ao observar o resultado abaixo, utilizando o mesmo input do algoritmo anterior da Maratona de Programação Paralela, vimos que optar por fazer uma ordenação com a primitiva sort garante uma melhora no desempenho da aplicação, tendo um tempo menor de **0m0.003s**, mesmo sem usar algum elemento de paralelismo.

In [17]:
%%writefile run_seq_sort.sh

#!bin/bash

# Compilando
! g++ seq_median_sort.cpp -o seq_median_sort

for i in {1..3}; do
  echo
  echo "run $i"
  time ./seq_median_sort input.in > /dev/null
done


Writing run_seq_sort.sh


In [19]:
! chmod +x run_seq_sort.sh;
! ./run_seq_sort.sh


run 1

real	0m0.003s
user	0m0.001s
sys	0m0.002s

run 2

real	0m0.003s
user	0m0.001s
sys	0m0.001s

run 3

real	0m0.002s
user	0m0.000s
sys	0m0.002s


# OpenMP


# GPU

Para o processamento em GPU utilizando CUDA, foi utilizados três estratégias diferentes para o cálculo da mediana. Todas as estratégias utilizavam a ideia de ordenar e depois pegar a mediana, tal que se o tamanho do vetor fosse ímpar a posição N/2 era escolhida como mediana e caso o tamanho do vetor fosse par a mediana seria a média entre a posição (N/2)-1 e (N/2). Entretanto, utilizamos diferentes técnicas de ordenação para conseguir apresentar estratégias diferentes para ordenar o nosso vetor.

1. **Primeira Estratégia: Diretivas da NVIDIA**

  A diretiva thrust da NVIDIA é uma biblioteca de programação paralela em C++ que facilita a implementação de algoritmos paralelos em GPUs NVIDIA usando CUDA. Ela oferece uma abstração de alto nível para simplificar o desenvolvimento de código paralelo e facilitar a portabilidade entre CPUs e GPUs.

  A função thrust::sort é utilizada para ordenar elementos em um vetor ou em uma sequência. A implementação exata do algoritmo de ordenação depende do tipo de dados e do contexto em que thrust::sort é aplicado. No entanto, em geral, a thrust pode usar diferentes algoritmos de ordenação, dependendo do tamanho do conjunto de dados e do tipo de hardware disponível.

  Para conjuntos de dados pequenos, a thrust pode usar algoritmos de ordenação baseados em comparação, como o mergesort ou quicksort. No entanto, para conjuntos de dados maiores, a thrust pode utilizar algoritmos de ordenação mais eficientes em termos de paralelismo, como o radix sort ou o merge sort paralelo.

2. **Segunda estratégia: Bitonic Sort**

  O Bitonic Sort é um algoritmo de ordenação paralela que foi projetado para ser executado eficientemente em arquiteturas paralelas, como GPUs (Graphics Processing Units). Esse algoritmo é especialmente adequado para sistemas que suportam operações de comparação em paralelo.

  O Bitonic Sort é baseado em uma sequência bitônica, que é uma sequência que começa em ordem crescente, atinge um ponto onde os elementos são os maiores e, a partir desse ponto, os elementos são ordenados em ordem decrescente. O algoritmo assume que a sequência a ser ordenada é uma sequência bitônica.

  A ideia central do Bitonic Sort é dividir repetidamente a sequência em subsequências bitônicas menores e, em seguida, combinar essas subsequências em uma sequência bitônica maior até que a ordenação seja concluída.


3. **Terceira estratégia: Merge Sort**

  O Mergesort é um algoritmo de ordenação eficiente e baseado na técnica de dividir para conquistar. Ele funciona dividindo repetidamente a lista não ordenada pela metade, ordenando cada metade e, em seguida, mesclando as metades ordenadas para criar uma lista ordenada. O processo de mesclagem é a chave para a eficiência do Mergesort.

## Primeira Estratégia


IPSIS FODASSE

In [ ]:
%%writefile thrust_median.cu

#include "cuda_runtime.h"
#include <cuda.h>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <algorithm>
#include <cstdlib>
#include <iostream>
#include <stdlib.h>




int main( int argc, char *argv[]) {

  //Standard parameters
  long int numValues = 1000000;


  if (argc == 2){
    numValues = atoi(argv[1]);
    printf("NUM VALUES SET TO %ld.\n",numValues);

  }

  else{
        printf("NUM_VALUES NOT SETTED\n");
        exit(1);
  }
  thrust::host_vector<int> h_vec;

  for (int i=numValues-1;i >= 0;i--)
    h_vec.push_back(i);

  // transfer data to the device
  thrust::device_vector<int> d_vec = h_vec;

  // sort data on the device (846M keys per second on GeForce GTX 480)
  thrust::sort(d_vec.begin(), d_vec.end());

  // transfer data back to host
  thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());


  double median = 0;

  if (h_vec.size() % 2 == 1){
   median = h_vec[(numValues-1)/2] ;
  }

  else
    median = (h_vec[(numValues/2)-1] + h_vec[numValues/2]) / 2.0;



    printf("The median is %.3lf\n",median);

  return 0;
}


Writing thrust_median.cu


## Segunda estratégia

PSI FODAS

In [ ]:
%%writefile bitonic_median.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <cuda.h>
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <algorithm>

#define MAX_THREADS_PER_BLOCK 256

//GPU Kernel Implementation of Bitonic Sort
__global__ void bitonicSortGPU_(int* arr, int j, int k)
{
    unsigned int i, ij;

    i = threadIdx.x + blockDim.x * blockIdx.x;

    ij = i ^ j;

    if (ij > i)
    {
        if ((i & k) == 0)
        {
            if (arr[i] > arr[ij])
            {
                int temp = arr[i];
                arr[i] = arr[ij];
                arr[ij] = temp;
            }
        }
        else
        {
            if (arr[i] < arr[ij])
            {
                int temp = arr[i];
                arr[i] = arr[ij];
                arr[ij] = temp;
            }
        }
    }
}

void bitonicSortGPU(int *arr,int size){

    int* gpuArrbiton;


    //Set number of threads and blocks for kernel calls
    int threadsPerBlock = MAX_THREADS_PER_BLOCK;
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;


    cudaMalloc((void**)&gpuArrbiton, size * sizeof(int));

    // Copy the input array to GPU memory
    cudaMemcpy(gpuArrbiton, arr, size * sizeof(int), cudaMemcpyHostToDevice);

    for (int k = 2; k <= size; k <<= 1)
    {
        for (int j = k >> 1; j > 0; j = j >> 1)

            bitonicSortGPU_<<<blocksPerGrid, threadsPerBlock >>> (gpuArrbiton, j, k);

    }

    cudaMemcpy(arr, gpuArrbiton, size * sizeof(int), cudaMemcpyDeviceToHost);
    return;


}
int main(int argc, char *argv[]){

  //Standard parameters
  long int numValues = 1000000;


  if (argc == 2){
    numValues = atoi(argv[1]);
    printf("NUM VALUES SET TO %ld.\n",numValues);

  }

  else{
        printf("NUM_VALUES NOT SETTED\n");
        exit(1);
  }

  //Create CPU based Arrays
  int* arr = new int[numValues];

int j=0;
  for (int i=numValues-1;i>=0;i--){
    arr[j] = i;
    j += 1;
  }

  bitonicSortGPU(arr,numValues);

  double median = 0;

  if (numValues % 2 == 1){
   median = arr[(numValues-1)/2] ;
  }

  else
    median = (arr[(numValues/2)-1] + arr[numValues/2]) / 2.0;



    printf("The median is %lf\n",median);



  return 0;


}


## Terceira estratégia

PSIS

In [ ]:
%%writefile merge_median.cu


#include <iostream>
#include <cstdio>
#include <algorithm>
#include <getopt.h>
#include <string.h>
#include <fstream>
#include <cuda.h>
#include <cuda_runtime.h>
#include <driver_functions.h>
#include <assert.h>
#include <thread>
#include <vector>




#define NUM_GPUS 3



__device__ void merge(int l,int m,int r,int data[],int tmp[])
{
    int i=l,j=m,k=l;
    while (i<m&&j<r)
    {
        if (tmp[i]<=tmp[j])
        {
            data[k++]=tmp[i++];
        }
        else
        {
            data[k++]=tmp[j++];
        }
    }
    while (i<m) data[k++]=tmp[i++];
    while (j<r) data[k++]=tmp[j++];
}



__global__ void
merge_kernel(int N, int chunk,int data[],int tmp[])
{
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    if (index>N) return ;
    //当数据量过多时，这里会发生溢出。暂时使用变负数。
    int start=index*chunk;
    if (start>=N || start<0) return ;

    int left=start;
    int mid=min(start+(int)(chunk/2),N);
    int right=min(start+chunk,N);
    // printf("l=%d,m=%d,r=%d\n",left,mid,right);
    // if (start<0) assert(0);
    merge(left, mid,right,data,tmp);
}

//cuda merge sort 就是chunk取2的幂次然后每个kernel会出里start+chunk的数据



void mergeSort(int N,int *input,int *output,int device=0)
{
    cudaSetDevice(device);

    int *device_i;
    int *tmp;
    cudaMalloc((void **)&device_i, N*sizeof(int));
    cudaMalloc((void **)&tmp, N*sizeof(int));


    cudaMemcpy(device_i, input, N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(tmp, input, N*sizeof(int), cudaMemcpyHostToDevice);



    for (int chunk=2;chunk<2*N;chunk*=2)
    {
        // const int threadsPerBlock = 512;
        const int threadsPerBlock=1;
        const int blocks = ((N + threadsPerBlock*chunk - 1) / (threadsPerBlock*chunk));
        merge_kernel<<<blocks,threadsPerBlock>>>(N,chunk,device_i,tmp);
        cudaDeviceSynchronize();
        merge_kernel<<<blocks,threadsPerBlock>>>(N,chunk,tmp,device_i);
        cudaDeviceSynchronize();
    }


    cudaMemcpy(output, device_i, N*sizeof(int), cudaMemcpyDeviceToHost);

    cudaError_t errCode = cudaPeekAtLastError();
    if (errCode != cudaSuccess) {
        fprintf(stderr, "WARNING: A CUDA error occured: code=%d, %s\n", errCode, cudaGetErrorString(errCode));
    }



    cudaFree(device_i);
    cudaFree(tmp);
}

void mergeSortMultiGPU(int N,int *input,int *output,int offset,int device=0)
{
    cudaSetDevice(device);

    int *device_i;
    int *tmp;
    cudaMalloc((void **)&device_i, N*sizeof(int));
    cudaMalloc((void **)&tmp, N*sizeof(int));


    cudaMemcpy(device_i, &input[offset], N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(tmp, &input[offset], N*sizeof(int), cudaMemcpyHostToDevice);



    for (int chunk=2;chunk<2*N;chunk*=2)
    {
        // const int threadsPerBlock = 512;
        const int threadsPerBlock=1;
        const int blocks = ((N + threadsPerBlock*chunk - 1) / (threadsPerBlock*chunk));
        merge_kernel<<<blocks,threadsPerBlock>>>(N,chunk,device_i,tmp);
        cudaDeviceSynchronize();
        merge_kernel<<<blocks,threadsPerBlock>>>(N,chunk,tmp,device_i);
        cudaDeviceSynchronize();
    }


    cudaMemcpy(output, device_i, N*sizeof(int), cudaMemcpyDeviceToHost);

    cudaError_t errCode = cudaPeekAtLastError();
    if (errCode != cudaSuccess) {
        fprintf(stderr, "WARNING: A CUDA error occured: code=%d, %s\n", errCode, cudaGetErrorString(errCode));
    }



    cudaFree(device_i);
    cudaFree(tmp);
}


void calculateElements(int *elementsPerGPU,int shards_num,long int vectorSize){

        //Define o tamanho de cada shard
    for (int i=0;i<shards_num;i++){

        if (i != (shards_num)-1){ elementsPerGPU[i] = vectorSize / (shards_num);}

        else{
            elementsPerGPU[i] = vectorSize;

            for (int j=0;j<i;j++)
                elementsPerGPU[i] -= elementsPerGPU[j];


        }
    }

    return;
}

void merging_partial_results(int *elementsPerGPU, int **partial_results,int numValues,int *results){

  int control[NUM_GPUS];

  for(int i=0;i<NUM_GPUS;i++)
    control[i] = 0;


  for(int i=0;i<numValues;i++){

    int menor = INT_MAX;
    int idx_menor = -1;

    for(int j=0;j<NUM_GPUS;j++){

      if (control[j] == elementsPerGPU[j]){continue;}


      else if (j == 0){ menor = partial_results[j][control[j]]; idx_menor = j;}

      else{ if (menor > partial_results[j][control[j]] ) {menor = partial_results[j][control[j]]; idx_menor = j;}}

    }
    control[idx_menor] += 1;
    results[i] = menor;
  }

  for(int i=0;i<NUM_GPUS;i++)
    printf("%d -> %d (%d)", i, control[i], elementsPerGPU[i]);



}

void printArray(int* A, int size)
{
	int i;
	for (i = 0; i < size; i++)
		printf("%d ", A[i]);
	printf("\n");
}

void printArray2(int* A, int size)
{
	int i;
	for (i = size-1; i < size-10; i++)
		printf("%d ", A[i]);
	printf("\n");
}

int main(int argc, char *argv[]){

  //Standard parameters
  long int numValues = 1000000;


  if (argc == 2){
    numValues = atoi(argv[1]);
    printf("NUM VALUES SET TO %ld.\n",numValues);

  }

  else{
        printf("NUM_VALUES NOT SETTED\n");
        exit(1);
  }

  //Create CPU based Arrays
  int* arr = new int[numValues];
  int*  result = new int[numValues];


  int j=0;
  for (int i=numValues-1;i>=0;i--){
    arr[j] = i;
    j += 1;
  }

  if (NUM_GPUS == 1){
    mergeSort(numValues,arr,result,0);
  }
  else{

    int *elements_per_GPU;
    elements_per_GPU = (int*)malloc(NUM_GPUS*sizeof(int));
    int *partial_results[NUM_GPUS];


    calculateElements(elements_per_GPU,NUM_GPUS,numValues);

    for(int i=0;i<NUM_GPUS;i++)
      partial_results[i] = new int[elements_per_GPU[i]];

    std::vector<std::thread> threads;

    for (int i=0;i<NUM_GPUS;i++){
      threads.push_back (std::thread ([elements_per_GPU, i,arr,partial_results] () {



        mergeSortMultiGPU(elements_per_GPU[i],arr,partial_results[i],i*elements_per_GPU[0],i);
      }));
  }

    for (auto &t: threads)
    t.join ();



  merging_partial_results(elements_per_GPU, partial_results,numValues,result);


  }


  double median = 0;

  if (numValues % 2 == 1){
   median = result[(numValues-1)/2] ;
  }

  else
    median = (result[(numValues/2)-1] + result[numValues/2]) / 2.0;

//  printArray(result,numValues);

    printf("The median is %f\n",median);

  return 0;

}
